# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np 
import pandas as pd 
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2207968769098,
 'week52high': 135.33,
 'week52low': 57.99,
 'week52change': 0.8672486140191357,
 'sharesOutstanding': 17604345886,
 'float': 0,
 'avg10Volume': 122033091,
 'avg30Volume': 108007918,
 'day200MovingAvg': 111.89,
 'day50MovingAvg': 120.63,
 'employees': 0,
 'ttmEPS': 3.42,
 'ttmDividendRate': 0.8396745430706233,
 'dividendYield': 0.006624100302570932,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-31',
 'nextEarningsDate': '0',
 'peRatio': 39.2710297382443,
 'beta': 1.1865985331625808,
 'maxChangePercent': 47.69341834638524,
 'year5ChangePercent': 4.249854279220026,
 'year2ChangePercent': 2.309613915422468,
 'year1ChangePercent': 0.8557523281962673,
 'ytdChangePercent': 0.74394227552501,
 'month6ChangePercent': 0.455188310616849,
 'month3ChangePercent': 0.19092040702853366,
 'month1ChangePercent': 0.07026582379673683,
 'day30ChangePercent': 0.06786351282690169,
 'day5ChangePercent': 0.03578872270962929}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.8557523281962673

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,price&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index=True
        )

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,123.500,0.42527,N/A
1,AAL,16.880,-0.433917,N/A
2,AAP,167.230,0.0313782,N/A
3,AAPL,127.529,0.836833,N/A
4,ABBV,107.190,0.252893,N/A
...,...,...,...,...
500,YUM,113.460,0.126704,N/A
501,ZBH,157.070,0.029611,N/A
502,ZBRA,389.000,0.492513,N/A
503,ZION,42.190,-0.178698,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace=True)
final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,39.600,2.2948,N/A
1,345,NVDA,545.650,1.32004,N/A
2,31,AMD,99.240,1.24326,N/A
3,387,PYPL,238.570,1.18502,N/A
4,275,LB,39.960,1.15246,N/A
...,...,...,...,...,...
500,328,NCLH,25.660,-0.546123,N/A
501,118,CTL,11.000,None,N/A
502,165,ETFC,50.040,None,N/A
503,326,MYL,15.885,None,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')

    try:
        float(portfolio_size)
    except:
        print("That is not a number! \n'Please try again: ")
        portfolio_size = input('Enter the size of your portfolio: ')

portfolio_input()
print(portfolio_size)

1000000


In [16]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.93,2.3027,50
1,345,NVDA,551.35,1.29739,3
2,31,AMD,100.5,1.27513,19
3,387,PYPL,248.21,1.18693,7
4,275,LB,39.21,1.17146,50
...,...,...,...,...,...
500,328,NCLH,25.67,-0.56517,77
501,118,CTL,11,None,180
502,165,ETFC,49.94,None,39
503,326,MYL,15.9,None,124


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [9]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
    ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,price&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index= hqm_columns
            ),
            ignore_index = True
        )
    
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [10]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0.0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.400,N/A,0.434254,0.877228,0.358017,0.740594,0.20654,0.677228,0.101368,0.819802,N/A
1,AAL,16.590,N/A,-0.429068,0.0257426,0.0324183,0.148515,0.252832,0.772277,0.29854,0.994059,N/A
2,AAP,162.100,N/A,0.0312817,0.427723,0.108424,0.289109,0.0433163,0.221782,0.0982118,0.809901,N/A
3,AAPL,128.193,N/A,0.832267,0.970297,0.459108,0.861386,0.187845,0.633663,0.0688552,0.689109,N/A
4,ABBV,105.130,N/A,0.242231,0.712871,0.113291,0.293069,0.17931,0.605941,0.0489911,0.6,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.100,N/A,0.125092,0.556436,0.260835,0.59802,0.20127,0.659406,0.0494675,0.60396,N/A
501,ZBH,153.400,N/A,0.0290649,0.425743,0.213124,0.491089,0.0990707,0.368317,0.0377896,0.524752,N/A
502,ZBRA,379.000,N/A,0.491694,0.906931,0.4311,0.827723,0.46944,0.956436,0.0424493,0.556436,N/A
503,ZION,41.820,N/A,-0.173166,0.150495,0.160576,0.372277,0.319579,0.869307,0.0721002,0.710891,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [11]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe
    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.400,N/A,0.434254,0.877228,0.358017,0.740594,0.20654,0.677228,0.101368,0.819802,0.778713
1,AAL,16.590,N/A,-0.429068,0.0257426,0.0324183,0.148515,0.252832,0.772277,0.29854,0.994059,0.485149
2,AAP,162.100,N/A,0.0312817,0.427723,0.108424,0.289109,0.0433163,0.221782,0.0982118,0.809901,0.437129
3,AAPL,128.193,N/A,0.832267,0.970297,0.459108,0.861386,0.187845,0.633663,0.0688552,0.689109,0.788614
4,ABBV,105.130,N/A,0.242231,0.712871,0.113291,0.293069,0.17931,0.605941,0.0489911,0.6,0.55297
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.100,N/A,0.125092,0.556436,0.260835,0.59802,0.20127,0.659406,0.0494675,0.60396,0.604455
501,ZBH,153.400,N/A,0.0290649,0.425743,0.213124,0.491089,0.0990707,0.368317,0.0377896,0.524752,0.452475
502,ZBRA,379.000,N/A,0.491694,0.906931,0.4311,0.827723,0.46944,0.956436,0.0424493,0.556436,0.811881
503,ZION,41.820,N/A,-0.173166,0.150495,0.160576,0.372277,0.319579,0.869307,0.0721002,0.710891,0.525743


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [41]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALGN,531.170,N/A,0.949927,0.986139,1.03898,0.990099,0.658344,0.994059,0.203533,0.958416,0.982178
1,FCX,24.880,N/A,0.945644,0.984158,1.3605,0.99802,0.455635,0.948515,0.171138,0.924752,0.963861
2,TWTR,57.890,N/A,0.74536,0.964356,0.675239,0.958416,0.394955,0.926733,0.285181,0.990099,0.959901
3,ALB,140.400,N/A,1.04839,0.988119,0.823722,0.978218,0.433583,0.940594,0.111083,0.849505,0.939109
4,XLNX,154.360,N/A,0.597241,0.932673,0.581076,0.936634,0.494772,0.964356,0.126734,0.889109,0.930693
5,PYPL,241.590,N/A,1.21383,0.994059,0.447633,0.851485,0.353358,0.89901,0.245339,0.978218,0.930693
6,LRCX,494.950,N/A,0.697907,0.950495,0.562207,0.934653,0.593262,0.982178,0.105194,0.833663,0.925248
7,AMD,97.370,N/A,1.2674,0.99604,0.784113,0.970297,0.280325,0.809901,0.122233,0.877228,0.913366
8,CDNS,132.590,N/A,0.927424,0.978218,0.453885,0.857426,0.335359,0.879208,0.174001,0.928713,0.910891
9,PAYC,456.180,N/A,0.692074,0.948515,0.387125,0.790099,0.616006,0.990099,0.122194,0.875248,0.90099


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [17]:
portfolio_input()

In [18]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.400,16,0.434254,0.877228,0.358017,0.740594,0.20654,0.677228,0.101368,0.819802,0.778713
1,AAL,16.590,119,-0.429068,0.0257426,0.0324183,0.148515,0.252832,0.772277,0.29854,0.994059,0.485149
2,AAP,162.100,12,0.0312817,0.427723,0.108424,0.289109,0.0433163,0.221782,0.0982118,0.809901,0.437129
3,AAPL,128.193,15,0.832267,0.970297,0.459108,0.861386,0.187845,0.633663,0.0688552,0.689109,0.788614
4,ABBV,105.130,18,0.242231,0.712871,0.113291,0.293069,0.17931,0.605941,0.0489911,0.6,0.55297
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.100,17,0.125092,0.556436,0.260835,0.59802,0.20127,0.659406,0.0494675,0.60396,0.604455
501,ZBH,153.400,12,0.0290649,0.425743,0.213124,0.491089,0.0990707,0.368317,0.0377896,0.524752,0.452475
502,ZBRA,379.000,5,0.491694,0.906931,0.4311,0.827723,0.46944,0.956436,0.0424493,0.556436,0.811881
503,ZION,41.820,47,-0.173166,0.150495,0.160576,0.372277,0.319579,0.869307,0.0721002,0.710891,0.525743


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [26]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [27]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [28]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column( f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [29]:
writer.save()